# Day 1

In [26]:
using System.Net;
using System;
using System.Net.Http;

async Task<string> GetInputAsync(int day)
{
    var url = $"https://adventofcode.com/2021/day/{day}/input";
    var secret = Environment.GetEnvironmentVariable("AOC_SECRET");
    if (string.IsNullOrEmpty(secret))
    {
        throw new InvalidOperationException("Need to set AOC_SECRET");
    }
    
    var cookieContainer = new CookieContainer();
    using var handler = new HttpClientHandler() { CookieContainer = cookieContainer };
    using var client = new HttpClient(handler);

    var cookie = new Cookie("session", secret);
    cookie.Domain = "adventofcode.com";
    cookieContainer.Add(cookie);
    var result = await client.GetAsync(url);
    return await result.Content.ReadAsStringAsync();
}

In [27]:
// Day 1

var input = await GetInputAsync(1);
var lines = input.Split("\n", StringSplitOptions.RemoveEmptyEntries);

// Part 1
var measurements = lines.Select(l => int.Parse(l)).ToArray();
var increases = 0;
for (var i = 1; i < measurements.Length; i++)
{
    if (measurements[i] > measurements[i-1])
    {
        increases++;
    }
}

Console.WriteLine($"Part 1: Incresed {increases} times");

// Part 2
var windowSize = 3;
increases = 0;
for (var i = 1; i < measurements.Length - (windowSize - 1); i++)
{
    var thisWindow = measurements[i..(i+3)];
    var previousWindow = measurements[(i-1)..(i+2)];
    if (thisWindow.Sum() > previousWindow.Sum())
    {
        increases++;
    }
}

Console.WriteLine($"Part 2: Incresed {increases} times");

Part 1: Incresed 1154 times
Part 2: Incresed 1127 times


# Day 2

In [28]:
// Day 2 

var input = await GetInputAsync(2);
var lines = input.Split("\n", StringSplitOptions.RemoveEmptyEntries);

var horizontal = 0;
var depth = 0;

foreach (var line in lines)
{
    var split = line.Split(" ");
    var command = split[0];
    var unit = split[1];
    var value = int.Parse(unit);
    switch (command.ToLower())
    {
        case "forward": horizontal += value; break;
        case "down": depth += value; break;
        case "up": depth -= value; break;
        default: throw new InvalidOperationException(command);
    }
}

Console.WriteLine($"Part 1: [{horizontal}, {depth}] => {horizontal * depth}");


// Part 2
horizontal = 0;
depth = 0;
var aim = 0;

foreach (var line in lines)
{
    var split = line.Split(" ");
    var command = split[0];
    var unit = split[1];
    var value = int.Parse(unit);
    switch (command.ToLower())
    {
        case "forward": horizontal += value; depth += aim * value; break;
        case "down": aim += value; break;
        case "up": aim -= value; break;
        default: throw new InvalidOperationException(command);
    }
}

Console.WriteLine($"Part 2: [{horizontal}, {depth}, {aim}] => {horizontal * depth}");

Part 1: [1832, 1172] => 2147104
Part 2: [1832, 1116059, 1172] => 2044620088


# Day 3

In [30]:
// Day 3
using System.Linq;

// Part 1
var input = await GetInputAsync(3);
var lines = input.Split("\n", StringSplitOptions.RemoveEmptyEntries);

var lineCount = lines.Length;

var bitLength = lines[0].Length;
var tracker = new int[bitLength];
var halfPoint = lineCount % 2 == 0 ? lineCount / 2 : (lineCount + 1) / 2;

foreach (var line in lines)
{
    var position = 0;
    foreach (var c in line)
    {
        if (c == '1')
        {
            tracker[position] += 1;
        }
        position++;
    }
}

// Construct string with most common bits for each value
var gammaString = "";
var epsilonString = "";
for (var i = 0; i < tracker.Length; i++)
{
    var sum = tracker[i];
    if (sum >= halfPoint)
    {
        gammaString += "1";
        epsilonString += "0";
    }
    else 
    {
        gammaString += "0";
        epsilonString += "1";
    }
}

var gammaRate = Convert.ToInt32(gammaString, 2);
var epsilonRate = Convert.ToInt32(epsilonString, 2);
Console.WriteLine($"Gamma: {gammaString} = {gammaRate}");
Console.WriteLine($"Epsilon: {epsilonString} = {epsilonRate}");
Console.WriteLine($"Answer: {gammaRate * epsilonRate}");


// Part 2
var oxygenStack = lines;
var co2Stack = lines;

// Reprocess under new rules
var oxygenString = "";
var co2String = "";

static (string[] ones, string[] zeroes) Split(string[] input, int spot)
{
    return (
        input.Where(v => v[spot] == '1').ToArray(),
        input.Where(v => v[spot] == '0').ToArray()
    );
}

for (var i = 0; i < bitLength; i++)
{
    if (oxygenStack.Length > 1)
    {
        var (ones, zeroes) = Split(oxygenStack, i);
        oxygenStack = ones.Length >= zeroes.Length ? ones : zeroes;
    }

    if (co2Stack.Length > 1)
    {
        var (ones, zeroes) = Split(co2Stack, i);
        co2Stack = ones.Length >= zeroes.Length ? zeroes : ones;
    }
}

var o2Value = Convert.ToInt32(oxygenStack.Single(), 2);
var c2Value = Convert.ToInt32(co2Stack.Single(), 2);

Console.WriteLine($"Oxygen: {oxygenStack.Single()} = {o2Value}");
Console.WriteLine($"CO2: {co2Stack.Single()} = {c2Value}");
Console.WriteLine($"Answer: {o2Value * c2Value}");

Gamma: 000010110001 = 177
Epsilon: 111101001110 = 3918
Answer: 693486
Oxygen: 001110100101 = 933
CO2: 111000100110 = 3622
Answer: 3379326


# Day 4

In [77]:
// Day 4
var input = await GetInputAsync(4);
var lines = input.Split("\n", StringSplitOptions.RemoveEmptyEntries);

class BingoNumber
{
    public int Number;
    public bool Marked; 

    public static BingoNumber Create(string s)
    {
        return new BingoNumber()
        {
            Number = int.Parse(s)
        };
    }
}

class BingoBoard
{
    public BingoNumber[][] Numbers;
    public bool IsWinning;
}

static BingoBoard GenerateBoard(string[] input)
{
    if (input.Length != 5)
    {
        throw new InvalidOperationException($"{input.Length} lines instead of 5");
    }

    var numbers = Enumerable.Range(0, 5).Select(i => 
    {
        var numbers = new BingoNumber[5];
        var splitLine = input[i].Split(" ", StringSplitOptions.RemoveEmptyEntries);
        for (var j = 0; j < 5; j++)
        {
            numbers[j] = BingoNumber.Create(splitLine[j]);
        }

        return numbers;
    }).ToArray();

    return new BingoBoard()
    {
        Numbers = numbers
    };
} 

static bool IsWinning(BingoBoard board)
{
    for (var i = 0; i < board.Numbers.Length; i++)
    {
        // Rows
        if (board.Numbers[i].All(n => n.Marked))
        {
            return true;
        }

        // Columns
        if (board.Numbers.All(c => c[i].Marked))
        {
            return true;
        }
    }

    return false;
}

static void PrintBoard(BingoBoard board)
{
    for (var i = 0; i < board.Numbers.Length; i++)
    {
        for (var j = 0; j < board.Numbers[i].Length; j++)
        {
            var value = board.Numbers[i][j];
            if (value.Marked)
            {
                Console.Write($"_{value.Number}");
            }
            else 
            {
                Console.Write(value.Number);
            }

            Console.Write("\t");
        }
        Console.WriteLine();
    }
}

static void MarkNumber(int number, BingoBoard board)
{
    for (var i = 0; i < board.Numbers.Length; i++)
    {
        for (var j = 0; j < board.Numbers[i].Length; j++)
        {
            var value = board.Numbers[i][j];
            if (value.Number == number)
            {
                value.Marked = true;
                return;
            }
        }
    }
}

var randomNumbers = lines[0].Split(",").Select(n => int.Parse(n)).ToArray();
var boardLines = lines[1..];
var boards = new List<BingoBoard>();

while (boardLines.Length > 0)
{
    boards.Add(GenerateBoard(boardLines[0..5]));
    boardLines = boardLines[5..];
}

BingoBoard winner = null;
var usedNumbers = new List<int>();
for (var i = 0; i < randomNumbers.Length; i++)
{
    usedNumbers.Add(randomNumbers[i]);

    foreach (var board in boards)
    {
        MarkNumber(randomNumbers[i], board);

        if (i > 5)
        {
            if (IsWinning(board))
            {
                winner = board;
                break;
            }
        }
    }

    if (winner != null)
    {
        break;
    }
}

### Part 1

In [79]:
Console.WriteLine(string.Join(",", usedNumbers));

Console.WriteLine("Winner is: ");
PrintBoard(winner);

var sumOfWinner = winner.Numbers.SelectMany(row => row).Where(n => !n.Marked).Sum(n => n.Number);
var answer = sumOfWinner * usedNumbers.Last();
Console.WriteLine($"Answer: {answer}");

99,56,7,15,81,26,75,40,87,59,62,24,58,34,78,86,44,65,18,94,20,17,98,29,57,92,14
Winner is: 
60	_14	37	_78	73	
80	_40	_58	30	64	
77	_92	_81	1	45	
79	_26	11	12	51	
25	_56	68	67	61	
Answer: 11774


### Part 2

In [84]:
winner.IsWinning = true;
var winners = new List<BingoBoard>();
winners.Add(winner);

var index = usedNumbers.Count();
while (winners.Count() < boards.Count())
{
    var losers = boards.Where(b => b.IsWinning == false);
    var number = randomNumbers[index++];
    usedNumbers.Add(number);

    foreach (var board in losers)
    {
        MarkNumber(number, board);
        if (IsWinning(board))
        {
            board.IsWinning = true;
            winners.Add(board);
        }
    }
}

var lastWinner = winners.Last();
var sumOfLastWinner = lastWinner.Numbers.SelectMany(row => row).Where(n => !n.Marked).Sum(n => n.Number);
var part2Answer = sumOfLastWinner * usedNumbers.Last();
Console.WriteLine($"Answer: {part2Answer}");

Answer: 4495
